# Training Materials: Detecting Change with 2-Time Period Data

## 1. Introduction: Change Detection Basics

* **What is Change Detection?** Change detection is the process of identifying differences in the state of an object or phenomenon by observing it at different times.  Its purposes include monitoring land cover changes, assessing damage from natural disasters, and tracking urban development. Applications range from deforestation monitoring and urban sprawl analysis to disaster assessment and agricultural monitoring.

* **Types of Change Detection:** Several approaches exist, including image differencing (subtracting pixel values), image ratioing (dividing pixel values), and classification-based methods (comparing land cover classifications).

* **Two-Time Period Analysis:** This training focuses on analyzing change between two specific points in time. Advantages include simplicity and clear visualization of change. Limitations include potential misinterpretation due to seasonal variations or short-term events.

## 2. Change Detection Methods

## Change detection analysis based on analysis of 2-time period data of ALOS-2/Palsa-2

In this section,By analyzing the HH/HV scattering intensities during the two periods with Change Vector Analysis, the procedure for detecting and analyzing changes will be practiced. In the previous section, we analyzed land cover and land use changes by comparing color tones and patterns in color composite images from year to year.

Change vector analysisis a method used to analyze changes in 2D data points and is an effective method for analyzing HH/HV changes in PALSAR-2. In this section, we will deepen our understanding of the procedures and interpretations by analyzing the 2007 and 2010 annual composite data.


### 2.1. Conversion from DN values to backscatter coefficients

In the previous section, the pixel value（DN; digital number）was used for visualization as is, but for rigorous data analysis, it is desirable to convert the pixel values ​​into backscattering coefficients. The conversion is done using the following formula:

$$ \gamma^{0} = 10 log_{10} <DN^2> + CF $$

Note that, <> represents averaging for noise reduction and -83.0 dB is used for CF. [Global 25m Resolution PALSAR-2/PALSAR Mosaic (Ver. 2.4.0) Dataset Instructions](https://www.eorc.jaxa.jp/ALOS/jp/dataset/pdf/DatasetDescription_PALSAR2_Mosaic_ver240_ja.pdf). A coding example is shown below.

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

def calibrate_ima(image):
    return 10 * np.log10(image ** 2) - 83.0

def ima_cal(file_path):
    with rasterio.open(file_path) as src:
        original = src.read(1).astype('float32')
        calibrated = calibrate_ima(original)
    return original, calibrated

hh_2007_path = "Alos_2_2007_HH.tif" 
hh_2010_path = "Alos_2_2010_HH.tif"
hv_2007_path = "Alos_2_2007_HV.tif"
hv_2010_path = "Alos_2_2010_HV.tif"

hh_2007_raw, hh_2007_cal = ima_cal(hh_2007_path)
hh_2010_raw, hh_2010_cal = ima_cal(hh_2010_path)
hv_2007_raw, hv_2007_cal = ima_cal(hv_2007_path)
hv_2010_raw, hv_2010_cal = ima_cal(hv_2010_path)

fig, axs = plt.subplots(4, 2, figsize=(12, 16))

axs[0, 0].imshow(hh_2007_raw, cmap='gray')
axs[0, 0].set_title('Year 2007 HH (Original)')
axs[0, 1].imshow(hh_2007_cal, cmap='gray')
axs[0, 1].set_title('Year 2007 HH (Calibrated)')

axs[1, 0].imshow(hh_2010_raw, cmap='gray')
axs[1, 0].set_title('Year 2010 HH (Original)')
axs[1, 1].imshow(hh_2010_cal, cmap='gray')
axs[1, 1].set_title('Year 2010 HH (Calibrated)')

axs[2, 0].imshow(hv_2007_raw, cmap='gray')
axs[2, 0].set_title('Year 2007 HV (Original)')
axs[2, 1].imshow(hv_2007_cal, cmap='gray')
axs[2, 1].set_title('Year 2007 HV (Calibrated)')

axs[3, 0].imshow(hv_2010_raw, cmap='gray')
axs[3, 0].set_title('Year 2010 HV (Original)')
axs[3, 1].imshow(hv_2010_cal, cmap='gray')
axs[3, 1].set_title('Year 2010 HV (Calibrated)')

for i in range(4):
    fig.colorbar(axs[i, 0].imshow(hh_2007_raw, cmap='gray'), ax=axs[i, 0], orientation='vertical', label='DN')
    fig.colorbar(axs[i, 1].imshow(hh_2007_cal, cmap='gray'), ax=axs[i, 1], orientation='vertical', label='Calibrated dB')

plt.tight_layout()
plt.show()

### 2.2. Image Differencing/Ratioing

These methods highlight change by subtracting or dividing pixel values between the two images. They are simple but sensitive to atmospheric effects.

In [ ]:
# Example: Image Differencing
import numpy as np
from osgeo import gdal

# Open images
image1 = gdal.Open("image1.tif")
image2 = gdal.Open("image2.tif")

# Read bands as arrays
band1 = image1.GetRasterBand(1).ReadAsArray()
band2 = image2.GetRasterBand(1).ReadAsArray()

# Calculate difference
difference = band2 - band1

# Save difference image (replace with your desired output)
driver = gdal.GetDriverByName("GTiff")
out_ds = driver.Create("difference.tif", image1.RasterXSize, image1.RasterYSize, 1, gdal.GDT_Float32)
out_ds.GetRasterBand(1).WriteArray(difference)
out_ds = None

### 2.3. Classification-Based Change Detection

This approach involves classifying both images and comparing the results.

    * **1. Image Classification:** Classify both images using a supervised classification algorithm and a pre-trained model (or train a new model). Use the *same* model for both images.

    * **2. Change Map Generation:** Compare the classifications to create a change map, showing transitions between classes (e.g., forest to non-forest).

In [ ]:
# Example: Change Map Generation
import numpy as np

# Assuming 'classification1' and 'classification2' are numpy arrays
# containing the classification results for both time periods

change_map = np.where(classification1 != classification2, 1, 0)

# 'change_map' will have 1 where the classifications differ and 0 where they are the same

### 2.4. HH/HV change analysis

 Calculate the difference in image data to analyze the change from 2007 to 2010 for each HH/HV.

In [ ]:
delta_bandhv_cal = hv_2010_cal - hv_2007_cal
delta_bandhh_cal = hh_2010_cal - hh_2007_cal

fig, axs = plt.subplots(1, 2, figsize=(12, 16))

axs[0].imshow(delta_bandhv_cal, cmap='gray')
axs[0].set_title('HV change (2010 - 2007)')
axs[1].imshow(delta_bandhh_cal, cmap='gray')
axs[1].set_title('HH change (2010 - 2007)')
plt.tight_layout()
plt.show()

### 2.5. Change vector analysis (CVA) Data analysis

The changes that appear in the difference image calculated in the previous section can be expressed as vectors in the two-dimensional space of HH/HV pairs for each pixel.The figure below shows an example where HV has decreased significantly while HH has decreased slightly. The following code cell calculates the length of the change vector at each pixel, expressed as magnitude, and the direction as angle.

In [ ]:
magnitude = np.sqrt(delta_bandhv_cal**2 + delta_bandhh_cal**2)
angle = np.arctan2(delta_bandhv_cal, delta_bandhh_cal) #Radians: When using trigonometric functions like arctan or atan2, the default output is often in radians, where 00 to 2𝜋2π radians (approximately 00 to 6.286.28) covers the full circular range of angles.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

norm_magnitude = mcolors.Normalize(vmin=0, vmax=10)

fig, axs = plt.subplots(1, 2, figsize=(18, 6))

im1 = axs[0].imshow(magnitude, cmap='hsv', norm=norm_magnitude)
axs[0].set_title("Magnitude of change")
cbar1 = fig.colorbar(im1, ax=axs[0])
cbar1.set_label("Intensity")

im2 = axs[1].imshow(angle, cmap='hsv')
axs[1].set_title("Direction (Angle of change)")
cbar2 = fig.colorbar(im2, ax=axs[1])
cbar2.set_label("Radians")

plt.tight_layout()
plt.show()

## 3. Post-Classification Processing and Analysis

* **Change Reporting:** Create a report summarizing the detected changes.